In [1]:
from tqdm import tqdm 
import torch
import torch.nn.functional as F
from torch import nn
from torch.utils.data import DataLoader
from utils.base import eval_accuracy
from utils.base import get_correct_predictions_subset
from utils.data import create_butterfly_dataset
from model.butterfly_classifier import DenseNet121
from algo.attacker import adversarial_generator
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [2]:
if torch.tensor([23]) != 1:
    print('aaa')

aaa


In [ ]:
indices = torch.argmax(matches, dim=1)
indices = indices + 1
indices[~matches.any(dim=1)] = 0

In [ ]:
indices[~matches.any(dim=1)] = 0
indices.size()

In [ ]:
matches = (idx == a).to(int)
# indices: (batch_size, 1)
indices = torch.argmax(matches, dim=1)
indices = indices + 1
indices[~matches.any(dim=1)] = 0

In [ ]:
(idx == 0).sum(dim=1)

In [ ]:
(idx[0] == a).nonzero(as_tuple = True)[0]

In [ ]:
a = idx[:, 0][4] 
def get_rank(x, indices):
    vals = x[range(len(x)), indices]
    return (x > vals[:, None]).long().sum(1)
get_rank(idx, a)

# (0) Download Butterfly 

In [ ]:
!kaggle datasets download -d gpiosenka/butterfly-images40-species

In [ ]:
import zipfile
with zipfile.ZipFile('butterfly-images40-species.zip', 'r') as zip_ref:
    zip_ref.extractall('data/butterfly')

# (1) Process Butterfly Data

In [2]:
trainset, testset, normal_mapping, reverse_mapping, sample_img_dataset = create_butterfly_dataset()
assert len(trainset) == 12594, 'Size of train set not match'
assert len(testset) == 500, 'Size of test set not match'

# (2) Import Classifier

In [3]:
model = DenseNet121(num_classes=len(normal_mapping)).to('cuda')
model.load_state_dict(torch.load('./model/states/butterfly_classifier.pth'))

<All keys matched successfully>

# (3) Evaluate Untargeted Adversarial Examples

We also subset the parts where the model could provide correct predictions to attack

In [4]:
model.eval()
accuracy, correct_subset = get_correct_predictions_subset(model, testset, batch_size=100)
print('Accuracy on test set is {:.4f}'.format(accuracy))
print('Number of correctly predicted samples:', len(correct_subset))

Accuracy on test set is 0.9640
Number of correctly predicted samples: 482


# (4) Parameter Definition

In [6]:
# parameters definition
batch_size = 32
query_limit = 30000 # max attack limit 
search_var = 1e-3 # amount to perturb the input image
sample_num = 50 # 2*sample_num for estimating gradient
bound = 0.1 # the l-infinity distance between the adversarial example and the input image
# partial information paramters + lebel-only parameters
epsilon = 0.5 # initial searching range from the target image
delta = 0.01 # rate to decrease epsilon
eta_max = 0.02 # maximum learning rate
eta_min = 0.01 # minimum learning rate
k = 5 # information access
# label-only parameter
mu = 0.001 # radius for sampling ball
m = 50 # 2*number of sample for proxy score
# test_loader = DataLoader(correct_subset, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(correct_subset, batch_size = 32, shuffle = False)
lr = 0.01

# (5) Query-Limited Setting

In [ ]:
success_count = 0
query_counts = []
adv_images = []
with torch.no_grad():
    for batch in test_loader:
        images = batch[0]
        target_classes = batch[1]
        adv_image_batch, query_count_batch = adversarial_generator(model, target_classes, images, 
                                                                 search_var, sample_num,
                                                                bound, lr, query_limit)
        query_counts.append(query_count_batch)
        adv_images.append(adv_image_batch)
        # count the number of success attack
        success_count += sum(query_count <= query_limit for query_count in query_count_batch)
        print(success_count)
        break

In [ ]:
adv_image_all = torch.concat(adv_images, dim = 0)
query_count_all = torch.concat(query_counts, dim = 0)
torch.save(adv_image_all, "adv_img.pt")
torch.save(query_count_all, "query.pt")

# (6) Partial-Information Setting

In [10]:
from algo.attacker import PIA_adversarial_generator
from algo.defender import PartialInfo
from tqdm import trange

success_count_PIA = 0
query_counts_PIA = []
adv_images_PIA = []

top_k_output_model = PartialInfo(model=model, k=k)
with torch.no_grad():
    for i in trange(len(testset)):
        
        images = testset[i][0].unsqueeze(0)
        # images = batch[0]
        adv_image_batch, query_count_batch = PIA_adversarial_generator(top_k_output_model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit)
        query_counts_PIA.append(query_count_batch)
        adv_images_PIA.append(adv_image_batch)
        # count the number of success attack
        success_count_PIA += sum(query_count <= query_limit for query_count in query_count_batch)

100%|████████████████████████████████████████████████████████████████████████████████| 500/500 [42:46<00:00,  5.13s/it]


In [14]:
adv_image_all_pia = torch.concat(adv_images_PIA, dim = 0)
query_count_all_pia = torch.concat(query_counts_PIA, dim = 0)
torch.save(adv_image_all_pia, "adv_img_PIA.pt")
torch.save(query_count_all_pia, "query_PIA.pt")

In [15]:
query_count_all_pia

tensor([4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000.,
        4000., 4000., 4000., 4000., 4000., 4000., 4000., 4000., 

# (7) Label-only Setting

In [ ]:
from algo.attacker import PIA_adversarial_generator
success_count_LO = 0
query_counts_LO = []
adv_images_LO = []
with torch.no_grad():
    for batch in test_loader:
        images = batch[0]
        adv_image_batch, query_count_batch = PIA_adversarial_generator(model, images, sample_img_dataset,
                                                                      epsilon, delta, search_var,
                                                                      sample_num, eta_max, eta_min,
                                                                      bound, k, query_limit, label_only = True, mu = mu, m = m)
        query_counts_LO.append(query_count_batch)
        adv_images_LO.append(adv_image_batch)
        # count the number of success attack
        success_count_LO += sum(query_count <= query_limit for query_count in query_count_batch)
        break

In [ ]:
success_count_LO

# Playground + random testing

In [ ]:
test_img, test_label = testset[3]
test_img = test_img.unsqueeze(0)
initial_img, initial_label = testset[111]
initial_img = initial_img.unsqueeze(0)

In [ ]:
print('Adversarial: predicted class is {}'.format(reverse_mapping[torch.argmax(adv_logits, dim=1).detach().cpu().numpy()[0]]))
print('Original: predicted class is {}'.format(reverse_mapping[torch.argmax(org_logits, dim=1).detach().cpu().numpy()[0]]))

print('Adversarial: logit of true class is {:.2f}'.format(adv_logits[0, test_label]))
print('Original: logit of true class is {:.2f}'.format(org_logits[0, test_label]))

print('Adversarial: probability of true class is {:.2f}'.format(F.softmax(adv_logits, dim=1)[0, test_label]))
print('Original: probability of true class is {:.2f}'.format(F.softmax(org_logits, dim=1)[0, test_label]))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming x_adv is a PyTorch tensor with shape [C, H, W]
# where C is the number of channels, H is the height, and W is the width.

# Move the tensor to the CPU and convert to a NumPy array
image_np1 = adv_img.squeeze().cpu().detach().numpy()

# If the image has more than one channel, transpose the dimensions from [C, H, W] to [H, W, C]
if image_np1.ndim == 3:
    # Transpose the image for plotting
    image_np1 = image_np1.transpose(1, 2, 0)

# If the image is in the range [0, 1], ensure it's scaled to [0, 255] if needed
if image_np1.max() <= 1.0:
    image_np1 = (image_np1 * 255).astype(np.uint8)


In [ ]:
image_np = test_img.squeeze().cpu().detach().numpy()

# If the image has more than one channel, transpose the dimensions from [C, H, W] to [H, W, C]
if image_np.ndim == 3:
    # Transpose the image for plotting
    image_np = image_np.transpose(1, 2, 0)

# If the image is in the range [0, 1], ensure it's scaled to [0, 255] if needed
if image_np.max() <= 1.0:
    image_np = (image_np * 255).astype(np.uint8)

# Plot the image
plt.imshow(image_np)
plt.axis('off')  # Hide the axis
plt.show()

In [ ]:
image_np2 = initial_img.squeeze().cpu().detach().numpy()

# If the image has more than one channel, transpose the dimensions from [C, H, W] to [H, W, C]
if image_np2.ndim == 3:
    # Transpose the image for plotting
    image_np2 = image_np2.transpose(1, 2, 0)

# If the image is in the range [0, 1], ensure it's scaled to [0, 255] if needed
if image_np2.max() <= 1.0:
    image_np2 = (image_np2 * 255).astype(np.uint8)

# Plot the image
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(image_np1)
plt.title('Adversarial Image_processing')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(image_np2)
plt.title('Original Image')
plt.axis('off')
